# Setup

## Import & Path Setting

In [13]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [15]:
args = {
    "train_path" : "/kaggle/input/dakonddd/train",
    "test_path": "/kaggle/input/dakonddd/test"
    }

## Hyperparameter Setting

In [16]:
"""
#set1
CFG = {
    'RESIZE':300,
    'EPOCHS':10000,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}
"""

"""
#set2
CFG = {
    "AUG":
        {'RESIZE':400,
        'CENTERCROP':300, #imgsize = 300
        },
    'EPOCHS':10000,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}
"""

#set3
CFG = {
    "AUG":
        {'RESIZE':500,
        'CENTERCROP':300, #imgsize = 300
        },
    'EPOCHS':10000,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}



In [17]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Metadata Loading & Encoding

In [18]:
all_img_list = glob.glob(args["train_path"]+"/*/*")
df = pd.DataFrame(columns=['img_path', 'label'])
df["img_path"] = all_img_list
df["label"] = [path.split("/")[-2] for path in all_img_list]
print(f'dataset 크기 : {len(df)}')
print("-------------------------------")
print(f'클래스별 이미지 갯수')
print(f'{df.label.value_counts()}')
print("-------------------------------")
print("train test 분할")
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])
print(f'train 갯수 : {len(train)}\nval 갯수 :{len(val)}')
df.sample(10)

dataset 크기 : 3457
-------------------------------
클래스별 이미지 갯수
훼손         1405
오염          595
걸레받이수정      307
꼬임          210
터짐          162
곰팡이         145
오타공         142
몰딩수정        130
면불량          99
석고수정         57
들뜸           54
피스           51
창틀,문틀수정      27
울음           22
이음부불량        17
녹오염          14
가구수정         12
틈새과다          5
반점            3
Name: label, dtype: int64
-------------------------------
train test 분할
train 갯수 : 2419
val 갯수 :1038


,img_path,label
837,/kaggle/input/dakonddd/train/훼손/683.png,훼손
3456,/kaggle/input/dakonddd/train/몰딩수정/42.png,몰딩수정
415,/kaggle/input/dakonddd/train/훼손/61.png,훼손
3128,/kaggle/input/dakonddd/train/면불량/21.png,면불량
2569,/kaggle/input/dakonddd/train/오염/389.png,오염
2702,/kaggle/input/dakonddd/train/오염/86.png,오염
1812,/kaggle/input/dakonddd/train/꼬임/100.png,꼬임
3216,/kaggle/input/dakonddd/train/오타공/56.png,오타공
2931,/kaggle/input/dakonddd/train/곰팡이/92.png,곰팡이
2547,/kaggle/input/dakonddd/train/오염/549.png,오염


In [19]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
train.head(5)

,img_path,label
1647,/kaggle/input/dakonddd/train/훼손/292.png,18
414,/kaggle/input/dakonddd/train/훼손/238.png,18
646,/kaggle/input/dakonddd/train/훼손/1339.png,18
924,/kaggle/input/dakonddd/train/훼손/829.png,18
630,/kaggle/input/dakonddd/train/훼손/1243.png,18


## CustomDataset

In [20]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

# DataPreprocessing

In [21]:

"""
#Set1
train_transform = A.Compose([
                            A.Resize(CFG['RESIZE'],CFG['RESIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['RESIZE'],CFG['RESIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
"""

"""
#Set2
train_transform = A.Compose([
                            A.Resize(CFG["AUG"]['RESIZE'],CFG["AUG"]['RESIZE']),
                            A.CenterCrop(p=1, height=CFG["AUG"]["CENTERCROP"], width=CFG["AUG"]["CENTERCROP"]),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG["AUG"]['RESIZE'],CFG["AUG"]['RESIZE']),
                            A.CenterCrop(p=1, height=CFG["AUG"]["CENTERCROP"], width=CFG["AUG"]["CENTERCROP"]),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
"""
#Set3
train_transform = A.Compose([
                            A.Resize(CFG["AUG"]['RESIZE'],CFG["AUG"]['RESIZE']),
                            A.CenterCrop(p=1, height=CFG["AUG"]["CENTERCROP"], width=CFG["AUG"]["CENTERCROP"]),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG["AUG"]['RESIZE'],CFG["AUG"]['RESIZE']),
                            A.CenterCrop(p=1, height=CFG["AUG"]["CENTERCROP"], width=CFG["AUG"]["CENTERCROP"]),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [22]:
print(train_transform)
#print(test_transform)

Compose([
  Resize(always_apply=False, p=1, height=500, width=500, interpolation=1),
  CenterCrop(always_apply=False, p=1, height=300, width=300),
  Normalize(always_apply=False, p=1.0, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=False),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})


In [23]:
#변환 잘 되는지 확인
_t = cv2.imread(all_img_list[0])
train_transform(image=_t)["image"].shape

torch.Size([3, 300, 300])

# DataLoader Define

In [24]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


# Model Define

- efficientnet_b3
- inputsize = $(3,300,300)$

In [25]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b3(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [26]:
def train(model, optimizer, train_loader, val_loader, scheduler, earlystop_patience,device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    stop_count = 0
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            stop_count = 0
        else:
            stop_count+=1
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}] Stop Count : [{stop_count}]')
    
        if stop_count >= earlystop_patience:
            return best_model
    
    return best_model

In [27]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [28]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', verbose=True)
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, 10, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth


  0%|          | 0.00/47.2M [00:00<?, ?B/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.32630] Val Loss : [0.85298] Val Weighted F1 Score : [0.71983] Stop Count : [0]


...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 00024: reducing learning rate of group 0 to 1.8750e-05.
Epoch [24], Train Loss : [0.00313] Val Loss : [0.96499] Val Weighted F1 Score : [0.81125] Stop Count : [9]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [25], Train Loss : [0.00081] Val Loss : [0.97386] Val Weighted F1 Score : [0.80762] Stop Count : [10]


## Inference

In [29]:
test = pd.read_csv("/kaggle/input/dacon-wallpaper2/test.csv")
test["img_path"] = test.img_path.str.replace("./test",args["test_path"])

In [30]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [31]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [32]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [33]:
submit = pd.read_csv('./sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: './sample_submission.csv'

In [ ]:
submit['label'] = preds

In [ ]:
submit.loc[submit['label'] == '0', 'label'] = '가구수정'
submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
submit.loc[submit['label'] == '3', 'label'] = '꼬임'
submit.loc[submit['label'] == '4', 'label'] = '녹오염'
submit.loc[submit['label'] == '5', 'label'] = '들뜸'
submit.loc[submit['label'] == '6', 'label'] = '면불량'
submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
submit.loc[submit['label'] == '8', 'label'] = '반점'
submit.loc[submit['label'] == '9', 'label'] = '석고수정'
submit.loc[submit['label'] == '10', 'label'] = '오염'
submit.loc[submit['label'] == '11', 'label'] = '오타공'
submit.loc[submit['label'] == '12', 'label'] = '울음'
submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == '15', 'label'] = '터짐'
submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
submit.loc[submit['label'] == '17', 'label'] = '피스'
submit.loc[submit['label'] == '18', 'label'] = '훼손'

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)